<a href="https://colab.research.google.com/github/NKASG/deep-learning-keras-tf-tutorial/blob/master/plant_diesease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import models, layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = image_dataset_from_directory('/content/drive/MyDrive/Download',shuffle=True, image_size=(256,256), batch_size=32 )

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())
    

Visualization of our data set

In [ ]:
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

i want to split the date into 3 categories which are training_data,validation_data,Test_data which are 80%, 10%, 10% respectively which by calculation with length of 682 the training_data = 546 , validation_data = 68 and Test_data = 68

In [ ]:
total_len = len(dataset)
total_len

In [ ]:
train_no = total_len * 0.8
train_no

In [ ]:
train_no = round(train_no)

In [ ]:
train_ds = dataset.take(train_no)
len(train_ds)

In [ ]:
remain = dataset.skip(train_no)
len(remain)

In [ ]:
val_no = total_len * 0.1
val_no

In [ ]:
val_no = round(val_no)

In [ ]:
val_ds = remain.take(val_no)
len(val_ds)

In [ ]:
test_ds = remain.skip(val_no)
len(test_ds)

Prefetch and cache is used when loadingg batches while GPU is loading a batch the GPU will be loading the next batch its help saving time when reading those images while shuffle is needed for the image to be choosen randomly

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Normalizing of data by dividing each images by 255 and uniform resizing of the image to 255x255

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(256,256),
  layers.experimental.preprocessing.Rescaling(1./255),
])

Data Augmentation this is needed when we want to address over fitting and to boost accuracy

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
input_shape = (32,256,256,3)

In [ ]:
    Model = keras.models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    
    layers.Dense(3, activation='softmax'),
    
])
     
   

In [ ]:
 Model.build(input_shape)

In [ ]:
Model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
Model.summary()

In [ ]:
history = Model.fit( train_ds, batch_size=32, validation_data=val_ds, verbose=1, epochs=10)

In [ ]:
scores = Model.evaluate(test_ds)

In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy', )
plt.xlabel('epoch')
plt.legend(['accuracy','val_acc'], loc= 'upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss','val_loss'], loc= 'upper left')
plt.show()

model prediction 

In [ ]:
import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = Model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(Model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

Implementing Gradio

In [ ]:
#!pip install gradio


In [ ]:
#import gradio as gr

In [ ]:
#def predict_image(img):
  #img_4d=img.reshape(-1,256,256,3)
 # prediction=Model.predict(img_4d)[0]
#  return {class_names[i]: float(prediction[i]) for i in range(3)}

In [ ]:
#image = gr.inputs.Image(shape=(256,256))
#label = gr.outputs.Label(num_top_classes=3)

#gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch()

In [ ]:
Model.save("/content/sample_data")

In [ ]:
pip install tensorflow-model-optimization

In [ ]:
!git clone https://github.com/google/qkeras.git 
import sys 
sys.path.append('qkeras') 
!pip install git+https://github.com/keras-team/keras-tuner.git 
!pip install tensorflow_model_optimization 
import tensorflow_model_optimization as tfmot

In [ ]:
pip install --user --upgrade tensorflow-model-optimization

In [ ]:
# To install dependencies on Ubuntu:
# sudo apt-get install bazel git python-pip
# For other platforms, see Bazel docs above.
git clone https://github.com/tensorflow/model-optimization.git
cd model-optimization
bazel build --copt=-O3 --copt=-march=native :pip_pkg
PKGDIR=$(mktemp -d)
./bazel-bin/pip_pkg $PKGDIR
pip install --user --upgrade $PKGDIR/*.whl

In [ ]:
 import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(Model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

q_aware_model.summary()

In [ ]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.save(f"../models/{model_version}")